<a href="https://colab.research.google.com/github/Aiswariyat/audio-transcription-project/blob/main/Audio_Transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets librosa torchaudio soundfile


In [5]:
from datasets import load_dataset

# Load the minds14 dataset
dataset = load_dataset("PolyAI/minds14", name="en-US", split="train[:5%]")




In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")


In [9]:
import torch

def preprocess_audio(batch):
    audio_array, sample_rate = torchaudio.load(batch["path"])
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    audio_array = resampler(audio_array).squeeze().numpy()
    input_values = processor(audio_array, sampling_rate=16000, return_tensors="pt").input_values[0]
    batch["input_values"] = input_values
    return batch



In [10]:
def transcribe(batch):
    input_values = torch.tensor(batch["input_values"]).unsqueeze(0)
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    batch["transcription"] = processor.decode(predicted_ids[0])
    return batch


In [11]:

dataset = dataset.map(preprocess_audio)
transcriptions = dataset.map(transcribe)


Map:   0%|          | 0/28 [00:00<?, ? examples/s]

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

In [13]:
with open("all_transcriptions.txt", "w") as f:
    for idx, item in enumerate(transcriptions):
        f.write(f"Transcription {idx + 1}:\n")
        f.write(item["transcription"] + "\n\n")
